<a href="https://colab.research.google.com/github/alexweltgeist/twitter_bitcoin/blob/main/Create_Trainingset_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create Trainingset for CNN**

Download Twetts, apply sentiment via Textblob and lable the tweets

In [1]:
!pip install twython    
!pip install textblob
!pip install tweet-preprocessor

In [7]:
from twython import TwythonStreamer
from twython import Twython
import pandas as pd
from textblob import TextBlob
import preprocessor as p

In [8]:
# My key and secret
CONSUMER_KEY = '...'
CONSUMER_SECRET = '...'
ACCESS_TOKEN = '...'
ACCESS_TOKEN_SECRET = '...'

In [32]:
# Functions for Streaming
#del(tweets, twitter_df)
tweets = [] 

class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        """
        data in Python dict representing a tweet
        """
        # We only want to collect English-language tweets
        if data.get('lang') == 'en':
            tweets.append(data)
            if (len(tweets) % 10) == 0:
              print(f"received tweet #{len(tweets)}")

		# Stop when we've collected enough
        if len(tweets) >= 10000:
            self.disconnect()
        return tweets

    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()


In [33]:
# Functions for data extraction
def tcontent(stream):
    # Preprocessing options used for p.clean() below
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)
    textList = []
    for tweet in tweets:   
        textList.append(p.clean(tweet['text']))
    return textList

def sentiment(stream):
    dateList = []
    for tweet in tweets:   
        dateList.append(TextBlob(tweet['text']).sentiment.polarity)
    return dateList

def tnames(stream):
    nameList = []
    for tweet in tweets:   
        nameList.append(tweet['user']['name'])
    return nameList

def tdate(stream):
    dateList = []
    for tweet in tweets:   
        dateList.append(tweet['created_at'])
    return dateList


# Define the filter criteria
selection = ['Bitcoin', 'bitcoin']


In [36]:
# Consume public tweets with selection and create pandas dataframe
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
					ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

stream.statuses.filter(track=selection)


received tweet #1680
received tweet #1690
received tweet #1700
received tweet #1710
received tweet #1720
received tweet #1730
received tweet #1740
received tweet #1750
received tweet #1760
received tweet #1770
received tweet #1780
received tweet #1790
received tweet #1800
received tweet #1810
received tweet #1820
received tweet #1830
received tweet #1840
received tweet #1850
received tweet #1860
received tweet #1870
received tweet #1880
received tweet #1890
received tweet #1900
received tweet #1910
received tweet #1920
received tweet #1930
received tweet #1940
received tweet #1950
received tweet #1960
received tweet #1970
received tweet #1980
received tweet #1990
received tweet #2000
received tweet #2010
received tweet #2020
received tweet #2030
received tweet #2040
received tweet #2050
received tweet #2060
received tweet #2070
received tweet #2080
received tweet #2090
received tweet #2100
received tweet #2110
received tweet #2120
received tweet #2130
received tweet #2140
received twee

In [37]:
twitter_df = pd.concat([pd.Series(tcontent(tweets)), 
                        pd.Series(sentiment(tweets))],
                        axis=1)
twitter_df.columns = ['text', 'sentiment']

# Get rid of Rows with no Polarity:
# twitter_df = twitter_df[twitter_df['sentiment'] != 0]


In [38]:
twitter_df.head(5)

,text,sentiment
0,@BillyM2k #Kishu #kishuinu @InuKishu @10Ronald...,0.00
1,RT @BitcoinMagazine: Elon Musk taking a honest...,0.60
2,"RT @Daily_Swap: Remember in 2008 and 2009, the...",0.15
3,Breaking news !!!sent 3.000 #doge got back 30....,0.10
4,RT @bullishcoinss: $500 Giveaway #Bitcoin 1 pe...,0.00


In [39]:
def create_lable(polarity):
    polarity = float(polarity)
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive"

In [40]:
# Convert from Sentiment to Lable
twitter_df['lable'] = twitter_df.sentiment.apply(create_lable)
twitter_df = twitter_df.drop(['sentiment'], axis=1)


In [41]:
twitter_df.head(5)

,text,lable
0,@BillyM2k #Kishu #kishuinu @InuKishu @10Ronald...,Neutral
1,RT @BitcoinMagazine: Elon Musk taking a honest...,Positive
2,"RT @Daily_Swap: Remember in 2008 and 2009, the...",Positive
3,Breaking news !!!sent 3.000 #doge got back 30....,Positive
4,RT @bullishcoinss: $500 Giveaway #Bitcoin 1 pe...,Neutral


In [42]:
# Save dataframe 
path_to_save = "/content"
twitter_df.to_csv(path_to_save + '/all_tweets_with_lable.csv', sep=",", index=False)                             
